# Tips dataset analysis

Description: Fundamentals of Data Analysis - assignment project, GMIT 2019. See README.md for more background info.

>Author: **Andrzej Kocielski**  
>Github: [andkoc001](https://github.com/andkoc001/)  
>Email: G00376291@gmit.ie, and.koc001@gmail.com

Date of creation: 23-09-2019

This Notebook should be read in conjunction with the corresponding README.md file at the assignment repository at GitHub: <https://github.com/andkoc001/Tips_dataset_analysis/>.

___

## Introduction

### The Analysis
This Notebook presents my analysis and interpretation of the _**tips dataset**_. This is done through the following:
1. descriptive statistics of the raw data,
2. graphical representation of the data - plots,
3. inference of the implicit information by application of selected tools, methods and algorithms used in data analytics.


### The data

It is a common custom to offer some small extra money - a tip - to the staff of a restaurant on top of the bill after a satisfactory service received. Although the tips are voluntary, and the amount of the tips is not (usually) stated, by convention it is often advised to leave as a certain percent (e.g. 10%) of the total bill for the meal and service. 

The _tips dataset_ (being analysed in this Notebook) is a representation of tips given in a restaurant. The dataset in question is a record (allegedly real and true) of tips given along with total bills of a restaurant customers collated by a waiter working in the restaurant for several weeks. 

The data is organised in a form of an array, where the dataset attributes (aka features) are organised in columns, and the observations (aka instances) - in rows. The dataset consists of several data categories describing tips received in connection to circumstances, such as day of the week, gender of the tipper, etc. The data set includes 244 data observations.

The _tips dataset_ is integrated with the [Seaborn](https://seaborn.pydata.org/) - an external Python package for data visualisation. This dataset from this source will be used in the subsequent analysis. The dataset, however, can be also obtained from other sources, for example as a .csv file from [here](http://vincentarelbundock.github.io/Rdatasets/datasets.html).

### Hypothesis

The dataset provokes to ask the following question: **Is there a consistent linear relationship between _total bill_ and _amount of tip_?**

Below we will try to find evidences either supporting or the opposite this hypothesis. I will also delve deeper into the question and seek whether I could distinguish such a relationship considering other parameters, such as among smokers and non-smokers. 

###  Assumptions

* The data is accurate.
* The gender and the smoker refers to the person in the group, who payed the bill and left the tip.
* Currency is irrelevant.
* Any other information that might be normally applicable to bill-tip relationship (e.g. country, legislation and regulations, time of the year, etc.) and is not included in the dataset is irrelevant.

___
## Setting up the environment

### Importing additional packages  

The analysis will be conducted using numerical tools, specifically **Python** programming language. On top of that, I am going to use add-on Python libraries - **NumPy** (for numerical calculations), **Pandas** (for convenient data structures - DataFrames), as well as visualisation libraries - **Matplotlib** (its pyplot sub-library) and **Seaborn**.

In [1]:
# numerical calculations libraries, assigning aliases
import numpy as np
import pandas as pd

# plotting and data visualisation libraries, assigning aliases
import matplotlib.pyplot as plt
import seaborn as sns

# below command will allow for the plots being displayed inside the Notebook, rather than in a separate screen.
%matplotlib inline

### Loading the data set

Getting the data from _Seaborn_ package and assigning it to variable `tips`.

In [2]:
# Loading the data set
tips = sns.load_dataset("tips")

### The dataset basic properties and its integrity check 

Prior to actual data analysis, it is a good practice to check the data integrity, that is whether there are any empty cells or corrupted data. We will use for this purpose the Pandas function `info()`, which checks if there is any `null` value in _any_ column. This function also checks data type for each column, as well as number of each data types and number of observations (rows).

In [ ]:
tips.info()

Hence, there are no 'empty' cells in the dataset, and the above looks good. Perhaps the _size_ variable could be change to int8 type, so that the memory usage could be optimised, but considering the small size of the dataset, I am going to leave it as is.

The initial checks show its integrity and I am satisfied to continue with further analysis.

___

## Data analysis

### 1. Descriptive statistical analysis

#### Sneak peek into raw data

Below are listed several first raws of data displayed. This listing allows to get initial impression on the dataset structure, as well as its attributes (columns) and data types of the variables.

In [ ]:
tips.head(3)

For a quick glimpse into numeric values and basic statistical description, it is convenient to call `describe()` method. As a result it will show for each numerical attribute (column) information such as number of occurrence, minimum, median (50%) and maximum values, mean and standard deviation.

In [ ]:
# calling describe method; percentiles will be limited to 50%, to keep the result compact; adapted from https://www.geeksforgeeks.org/python-pandas-dataframe-describe-method/
  
tips.describe(percentiles = [0.5])

Interesting to see that tips were given every time. Also, the percent of mean tips is $13.16\%$ (=2.998279÷(19.785943+2.998279)).

Now, let's also see what are the unique values occur in each non-numeric (categorical) column, using Pandas' method `unique()`.

In [ ]:
print("sex:\t", sorted(tips.sex.unique()))
print("smoker:\t", sorted(tips.smoker.unique()))

In [ ]:
# List unique values in the categorical atributes
# Adapted from https://chrisalbon.com/python/data_wrangling/pandas_list_unique_values_in_column/ and https://jovianlin.io/data-visualization-seaborn-part-1/.

print("Values occuring in column 'sex':\t", sorted(tips.sex.unique()))
print("Values occuring in column 'smoker':\t", sorted(tips.smoker.unique()))
print("Values occuring in column 'day':\t", sorted(tips.day.unique()))
print("Values occuring in column 'time':\t", sorted(tips.time.unique()))

#### Raw data modeling

It would be interesting to consider and evaluate the relationship between the data which is not explicitly given. For this purpose, let's model the dataset by creating new attributes. I am going to create the following:
- `tip_ratio` - ratio of tip to bill,
- `sum` - the sum of total bill and tip,
- `percent` - tip as a percent of sum,
- `bpp` - amount of total bill divided by number of people in the group,
- `tpp` - amount of tip per person.

In [9]:
# new column created - percent of tip
tips["tip_ratio"] = tips["tip"]/tips["total_bill"] 

# new column created - sum of total bill and tip
tips["sum"] = tips["total_bill"]+tips["tip"] # appended at the end of the array

# new column created - ratio of tip to sum
tips["percent"] = round(tips["tip"]/tips["sum"]*100, 2)

# add column: bpp - bill per person
tips["bpp"] = tips["total_bill"]/tips["size"]

# add column: tpp - tip per person
tips["tpp"] = tips["tip"]/tips["size"]


Now, let's see what the dataset array looks like with the additional attributes. To keep it compact, just a few first rows.

In [10]:
tips.head(2)

,total_bill,tip,sex,smoker,day,time,size,tip_ratio,sum,percent,bpp,tpp
0,16.99,1.01,Female,No,Sun,Dinner,2,0.059447,18.0,5.61,8.495000,0.505000
1,10.34,1.66,Male,No,Sun,Dinner,3,0.160542,12.0,13.83,3.446667,0.553333


Let's use the `describe()` method again for the upgraded data array.

In [11]:
# calling describe method; percentiles will be limited to 50%, to keep the result compact
tips.describe(percentiles = [0.5])

,total_bill,tip,size,tip_ratio,sum,percent,bpp,tpp
count,244.000000,244.000000,244.000000,244.000000,244.000000,244.000000,244.00000,244.000000
mean,19.785943,2.998279,2.569672,0.160803,22.784221,13.643320,7.88823,1.212762
std,8.902412,1.383638,0.951100,0.061072,9.890116,4.058544,2.91435,0.491705
min,3.070000,1.000000,1.000000,0.035638,4.070000,3.440000,2.87500,0.400000
50%,17.795000,2.900000,2.000000,0.154770,20.600000,13.405000,7.25500,1.107500
max,50.810000,10.000000,6.000000,0.710345,60.810000,41.530000,20.27500,3.333333


From the above we can get some interesting observations:
1. on average for the entire dataset, the percent of tip is $13.64\%$ (which is slightly higher than the value calculated above for mean tip and mean total bill),
2. on average, total bill split equally in the group is 7.89,
3. on average, tip per person is 1.21.

For comparison, I am going to apply a filter and see what is, for example, the average ratio tip to total bill among smokers.

In [16]:
# mean tip per smoking person
print("Average tip per smoking person:\t\t\t", tips[tips["smoker"] == "Yes"]["tpp"].mean()) # filtering by applying conditional check (smoker=yes)

# mean total bill among smokers
print("Average total bill among smokers:\t\t", tips[tips["smoker"] == "Yes"]["total_bill"].mean())

# mean ratio among smokers
tips["ratio_smoke"] = tips[tips["smoker"] == "Yes"]["tip"] / tips[tips["smoker"] == "Yes"]["total_bill"]
print("Average ratio tip to total bill among smokers:\t", tips["ratio_smoke"].mean())

Average tip per smoking person:			 1.2977956989247312
Average total bill among smokers:		 20.756344086021507
Average ratio tip to total bill among smokers:	 0.1631960446368779


Interesting. It turns out, the smokers are more generous and pay more for the service as well as give higher tips. On average the ratio tip to total bill among smokers is $16.32\%$.

### 2. Graphical interpretation 

Some information can be revealed easily by visualising the data graphically on carefully crafted plots, according to the popular saying "A single plot is worth a thousand of data points". 

In this section, we will first look at the raw data in the set. The attempt made here is to visualise the figures, applying only built-in analytical tools. My intention is to draw attention to some interesting (in my view) and yet rather straightforward relationships between the data. Later on, I will try to apply more sophisticated analytical tools - specifically liner regression, with aim to reveal some less obvious relationships and patterns existing in the dataset.

For a better readability, let's change the Seaborn global plots style as follows (https://seaborn.pydata.org/tutorial/aesthetics.html, https://www.datacamp.com/community/tutorials/seaborn-python-tutorial)

In [ ]:
sns.set_style("darkgrid") # plot style
sns.set_palette("muted") # palette of colours

# reference: https://seaborn.pydata.org/tutorial/color_palettes.html, https://stackoverflow.com/a/47955814

#### Bar plots

Bar plots (also histograms, although not being the same class of graphs, [reference](https://keydifferences.com/difference-between-histogram-and-bar-graph.html)) allow to easily display the categorical variables and showing their distributions, especially in comparison with one or more similar categories of data. 

To begin with, let us see the tips distribution on a histogram, using `distplot` method. With a large number of bins, the tips intervals will be more precise, and it still clearer than `countplot` method.

In [ ]:
# Display setting
plt.subplots(figsize=(12,3))
plt.legend(prop={'size': 10})
plt.suptitle("Figure 1. How many times the same tip amount was offered.", y=1.0, fontsize=14)
#plt.xlabel('Tip')
plt.ylabel('Count')

# plot data and properties
sns.distplot(tips['tip'], bins=100, kde=False, color="k")

plt.show()

From the above figure (Fig.1) we can see that distinctly the most popular tip is 2 (units of currency), with a runner up at 3 (units of currency).

In the below figur (Fig. 2), we will have two plots, showing the distribution of smokers and non-smokers among all the customers on the first plot, and the distribution of male and female customers. For this purpose, we will use `seaborn.countplot()` function ([Seaborn documentation](https://seaborn.pydata.org/generated/seaborn.countplot.html)). 

In [ ]:
# two plots on one figure
fig, ax = plt.subplots(ncols=2, figsize=(12, 2))
plt.suptitle("Figure 2. Quantitative comparison of the clients.", y=1.1, fontsize=14) # https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.pyplot.suptitle.html
plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=.3, hspace=None)

# plot 1 - amount of smokers and non-smokers
sns.countplot(data=tips, y="day", ax=ax[0]) # placing the categories (smoker in this case) on y-axis makes the bars oriented horizontally

# plot #2 - amount of male and females
sns.countplot(data=tips, y="time", hue="smoker", ax=ax[1]) # placing the categories (gender in this case) on y-axis makes the bars oriented horizontally
plt.legend(bbox_to_anchor=(1.01, 1), loc=2, borderaxespad=0.) # legend put outside of the plot area

fig.show()

The below plots show the number of customers split onto days the restaurant was open. The second plot is further broken down by the size of the party (colour coded - see the legend).

In [ ]:
# Display setting
fig, ax = plt.subplots(ncols=2, figsize=(12, 2))
plt.suptitle("Figure 3. Number of clients on each day.", y=1.1, fontsize=14)
plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=.3, hspace=None)


# plot data and properties
sns.countplot(data=tips, y="size", ax=ax[0]) # number of clients on each day
sns.countplot(data=tips, y="smoker", hue="sex", ax=ax[1]) # additional discrimination on party sizy
plt.legend(bbox_to_anchor=(1.01, 1), loc=2, borderaxespad=0.) # Put the legend out of the figure, taken from https://stackoverflow.com/a/34579525

plt.show()



Barplot, below, is a convenient tool for illustrating categorical data. The top (or length for horizontal bars) of the bar represents the _mean_ (in the below plots, default) of the category (feature). _Seaborn_ allows also to show _confidence interval_ of the feature. Again, the default value of 95% is used in the plots below. The plots will be generated using the `seaborn.barplot()` function ([Seaborn docs](https://seaborn.pydata.org/generated/seaborn.barplot.html)).

Below, we see amount of tip offered each day, further broken down on whether the client was a smoker or not.

In [ ]:
# adapted from https://medium.com/@neuralnets/data-visualization-with-python-and-seaborn-part-1-29c9478a8700
fig, ax = plt.subplots(ncols=1, figsize=(6,3))
plt.title('Figure 4. Mean amount of tips on each day.')

# Mean amount of tip on each day
sns.barplot(data = tips, x = "day", y = "tip", hue="smoker")
plt.legend(bbox_to_anchor=(1.01, 1), loc=2, borderaxespad=0.) # legend put outside of the plot area

plt.show()

#### Scatter plots  

Scatter plots allow for relating two (or more) numerical variables. In the below plots, function `seaborn.relplot()` is used, with the default kind of the plot (scatter),  to show statistical relationship between variables. The values on x-axis are describe the total bill; y-axis - amount of tip received. This section is based on official Seaborn [tutorial](https://seaborn.pydata.org/tutorial/relational.html).


In [ ]:
sns.relplot(data=tips, x="total_bill", y="tip", s=20, height=4, aspect=1)
plt.title("Figure 5. Total bill to amount of tip.")
plt.show()

The above plot may appear over crowded with data points. By adding another variable (for example _smoker_) it may become clearer and unveil a potential pattern, further separated by days.

In [ ]:
fig, ax = plt.subplots(ncols=2, sharey=True)
plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=None)

# plot #1 - Smokers
sns.swarmplot(x="total_bill", y="tip", hue="smoker", marker='.', data=tips[tips["smoker"] == "Yes"], ax=ax[0])
# plot #2 - Non-smokers
sns.swarmplot(x="total_bill", y="tip", hue="smoker", marker='.', data=tips[tips["smoker"] == "No"], ax=ax[1])

fig.show()

In [ ]:
sns.relplot(data=tips, x="total_bill", y="tip", col="day", row="smoker", hue="smoker", s=20, height=4, aspect=1)

plt.show()

Except for Sunday smokers plot, which looks a bit random, there seems to be a distinct relationship between the total bill and the amount of tip.

What about the amount of tips compared to the size of the party on given days? Can we see any pattern there? We will apply `catplot()` method to generate plots.

In [ ]:
sns.catplot(data=tips, x="size", y="tip", col="day", s=4, height=4, aspect=1)
plt.show()

From the above plots, it appear there indeed is a relationship.

### 3.1 Linear regression

#### Minimising the cost

This part is based on the lecture concerning linear regression and the lecturer's [Notebook](https://github.com/ianmcloughlin/jupyter-teaching-notebooks/raw/master/simple-linear-regression.ipynb).

In this section I will test the hypothesis question stated above (what is the relationship (if any) between total bill and amount of tips), by estimating a regression line between the variables. I will see if there is a relationship between the amount of tip and total bill, which could be accurately described with $ y = m x + 1 $ equation. "The number $ m $ is the slope of the line. The slope is how much $ y $ increases by when $ x $ is increased by 1.0. The number $ c $ is the y-intercept of the line."

Taken from [Notebook](https://github.com/ianmcloughlin/jupyter-teaching-notebooks/raw/master/simple-linear-regression.ipynb):

>We want to calculate values of $m$ and $c$ that give the lowest value for the cost value above.
For our given data set we can plot the cost value/function.
Recall that the cost is:  
> $$ Cost(m, c) = \sum_i (y_i - mx_i - c)^2 $$
>This is a function of two variables, $m$ and $c$, so a plot of it is three dimensional. (...)  
>In the case of fitting a two-dimensional line to a few data points, we can easily calculate exactly the best values of $m$ and $c$. (...)  
We first calculate the mean (average) values of our $x$ values and that of our $y$ values.
Then we subtract the mean of $x$ from each of the $x$ values, and the mean of $y$ from each of the $y$ values.
Then we take the *dot product* of the new $x$ values and the new $y$ values and divide it by the dot product of the new $x$ values with themselves.
That gives us $m$, and we use $m$ to calculate $c$.  

We will use `polyfit()` method to calculate $m$ and $c$.

In [ ]:
line_eq = np.polyfit(tips["total_bill"], tips["tip"], 1) # fiting a stright line (first degree); assign to a list type

# m is stored in the first item of line_eq (index 0),
# c is stored in the second item of line_eq (index 1).

# Printing results with comment
print("Value of m is", line_eq[0], ", and value of c is", line_eq[1],".")

Putting the scatter plot and the calculated line in one plot.

In [ ]:
fig, ax = plt.subplots(ncols=2, figsize=(16,3))
plt.suptitle('Figure xxx, Overlay', x=0.5, y=1.05, fontsize = 14)
plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=0.1, hspace=None)

# plot #1 - 
sns.relplot(data=tips, x="total_bill", y="tip", s=20, ax=ax[0])
#ax[0].set_title('Relplot function')

# plot #2 - 
x = np.linspace(tips["total_bill"].min(), tips["total_bill"].max(), 2) # 2 of intermediate steps of 'x' in boundries between minimum and maximum 'total_bill' values
ax[0].plot(x, line_eq[0]*x + line_eq[1], 'r-'); # drawing the regression line with calculated minimum cost
#sns.lineplot(data=(line_eq[0]*x + line_eq[1]), palette=("RdBu"), ax=ax[0])

# Set some reasonable plot limits.
#plt.xlim([-1, 50])
#plt.ylim([-1, 10])

plt.show()

#### Seaborn built-in functionality

Based on the Seaborn [tutorial](https://seaborn.pydata.org/tutorial/regression.html).

Seaborn module offers a built-on functionality for drawing linear regression line automatically. Once again (compare to Fig. 5 above), let's have a look at a plot showing total bill (x-axes) against tip. This time, using `seaborn.regplot()` method, and with additional regression line to the plot.

In [ ]:
sns.regplot(data=tips, x="total_bill", y="tip", ci=0, scatter_kws={"s": 3})
sns.set(rc={'figure.figsize':(4,4)}) # plot size

The above plot indeed seem to show a linear relationship between the two variables. Before verifying that (calculation the minimum _cost_ value), let as look at other, arguably more interesting plots that can be produced from the dataset.

First, let us now consider the data with variables discrimination on day and whether the client was a smoker or not.

In [ ]:
lm = sns.lmplot(data=tips, x="total_bill", y="tip", col="day", row="smoker", hue="smoker", height=4, aspect=1, scatter_kws={"s": 10})

# axes limits; adapded from https://stackoverflow.com/a/25213438
axes = lm.axes
axes[0,0].set_ylim(0,11)
axes[0,0].set_xlim(0,55)

plt.show()

An interesting results yield the next set of plots, showing a tendency of decrease tip amount per person for larger group of people.

In [ ]:
lm = sns.lmplot(data=tips, x="total_bill", y="tpp", col="size", row="smoker", hue="smoker", height=4, aspect=.75, scatter_kws={"s": 4})

axes = lm.axes
axes[0,0].set_ylim(0,11)
axes[0,0].set_xlim(0,55)

plt.show()

___
### 3.2 k-nearest neighbours algorithm

This section is based on the Programming for Data Analysis, GMIT, lecture videos and the lecturer's [Notebook](https://github.com/ianmcloughlin/jupyter-teaching-notebooks/raw/master/pandas-with-iris.ipynb).  

Other consulted references:  
[Machine Learning Basics with the K-Nearest Neighbors Algorithm](https://towardsdatascience.com/machine-learning-basics-with-the-k-nearest-neighbors-algorithm-6a6e71d01761)  
[K-Nearest Neighbors Algorithm Using Python](https://www.edureka.co/blog/k-nearest-neighbors-algorithm/)
[K-Nearest Neighbors Algorithm in Python and Scikit-Learn](https://stackabuse.com/k-nearest-neighbors-algorithm-in-python-and-scikit-learn/)

_k-nearest neighbours_ is a supervised machine learning algorithms. It is used to solve a classification problem. It produces a discreet output (that is: either this or that class (there may be more classes), but not something in between). 

#### Importing SciKit Learn Library

In [ ]:
import sklearn.neighbors as nei
import sklearn.model_selection as mod

In [ ]:
tips.head(2) # shape reminder: (244, 7)

#### A glimpse into plot.  
The below plot - relationship between tip size, total bill and the smoker - is deemed the most suitable for my analysis. The other variables combinations produce more fuzzy plots (a lot of overlapping data points).

In [ ]:
#sns.pairplot(tips, hue="smoker")
#plt.show()

Assigning `Inputs` and `Outputs`

In [ ]:
inputs = tips[['total_bill', 'tip']]
outputs = tips['sex']

Classifier `knn`

In [ ]:
knn = nei.KNeighborsClassifier(n_neighbors=5) # will consider 5 nearest neighbours

Applying `fit` function

In [ ]:
knn.fit(inputs, outputs)

Evaluation of the results

In [ ]:
(knn.predict(inputs) == outputs).sum() # Returns number of correctly recognised samples; total number of samples is 244

Training and testing data sub-sets.

The dataset is split randomly into:  
1) training (75% of entire dataset size, i.e. 183), and  
2) testing (25%, i.e. 61)

In [ ]:
inputs_train, inputs_test, outputs_train, outputs_test = mod.train_test_split(inputs, outputs, test_size=0.25)

In [ ]:
knn = nei.KNeighborsClassifier(n_neighbors=5)
knn.fit(inputs_train, outputs_train)

In [ ]:
# knn.predict(inputs_test) == outputs_test

In [ ]:
answer = (knn.predict(inputs_test) == outputs_test).sum()
answer

Hence, the accuracy of the algorithm is the ratio of correctly recognised to actual number, and is shown below.

In [ ]:
print("%.02f" % ((answer/61) * 100), "%")

___
## Findings and conclusion

As we can see from the above plots, there seem to be a strong linear relationship between the amount of tip and the total bill for the entire dataset as well as for a number of sub-sets.

___
## References

___
Andrzej Kocielski